In [7]:
import pandas as pd 
import numpy as np
from IPython.core.display import HTML
from string import Template
import json
import os
import random
import math

In [8]:
HTML('''
<script src="../../sigma.js/src/sigma.core.js"></script>
<script src="../../sigma.js/src/conrad.js"></script>
<script src="../../sigma.js/src/utils/sigma.utils.js"></script>
<script src="../../sigma.js/src/utils/sigma.polyfills.js"></script>
<script src="../../sigma.js/src/sigma.settings.js"></script>
<script src="../../sigma.js/src/classes/sigma.classes.dispatcher.js"></script>
<script src="../../sigma.js/src/classes/sigma.classes.configurable.js"></script>
<script src="../../sigma.js/src/classes/sigma.classes.graph.js"></script>
<script src="../../sigma.js/src/classes/sigma.classes.camera.js"></script>
<script src="../../sigma.js/src/classes/sigma.classes.quad.js"></script>
<script src="../../sigma.js/src/classes/sigma.classes.edgequad.js"></script>
<script src="../../sigma.js/src/captors/sigma.captors.mouse.js"></script>
<script src="../../sigma.js/src/captors/sigma.captors.touch.js"></script>
<script src="../../sigma.js/src/renderers/sigma.renderers.canvas.js"></script>
<script src="../../sigma.js/src/renderers/sigma.renderers.webgl.js"></script>
<script src="../../sigma.js/src/renderers/sigma.renderers.svg.js"></script>
<script src="../../sigma.js/src/renderers/sigma.renderers.def.js"></script>
<script src="../../sigma.js/src/renderers/webgl/sigma.webgl.nodes.def.js"></script>
<script src="../../sigma.js/src/renderers/webgl/sigma.webgl.nodes.fast.js"></script>
<script src="../../sigma.js/src/renderers/webgl/sigma.webgl.edges.def.js"></script>
<script src="../../sigma.js/src/renderers/webgl/sigma.webgl.edges.fast.js"></script>
<script src="../../sigma.js/src/renderers/webgl/sigma.webgl.edges.arrow.js"></script>
<script src="../../sigma.js/src/renderers/canvas/sigma.canvas.labels.def.js"></script>
<script src="../../sigma.js/src/renderers/canvas/sigma.canvas.hovers.def.js"></script>
<script src="../../sigma.js/src/renderers/canvas/sigma.canvas.nodes.def.js"></script>
<script src="../../sigma.js/src/renderers/canvas/sigma.canvas.edges.def.js"></script>
<script src="../../sigma.js/src/renderers/canvas/sigma.canvas.edges.curve.js"></script>
<script src="../../sigma.js/src/renderers/canvas/sigma.canvas.edges.arrow.js"></script>
<script src="../../sigma.js/src/renderers/canvas/sigma.canvas.edges.curvedArrow.js"></script>
<script src="../../sigma.js/src/renderers/canvas/sigma.canvas.edgehovers.def.js"></script>
<script src="../../sigma.js/src/renderers/canvas/sigma.canvas.edgehovers.curve.js"></script>
<script src="../../sigma.js/src/renderers/canvas/sigma.canvas.edgehovers.arrow.js"></script>
<script src="../../sigma.js/src/renderers/canvas/sigma.canvas.edgehovers.curvedArrow.js"></script>
<script src="../../sigma.js/src/renderers/canvas/sigma.canvas.extremities.def.js"></script>
<script src="../../sigma.js/src/renderers/svg/sigma.svg.utils.js"></script>
<script src="../../sigma.js/src/renderers/svg/sigma.svg.nodes.def.js"></script>
<script src="../../sigma.js/src/renderers/svg/sigma.svg.edges.def.js"></script>
<script src="../../sigma.js/src/renderers/svg/sigma.svg.edges.curve.js"></script>
<script src="../../sigma.js/src/renderers/svg/sigma.svg.labels.def.js"></script>
<script src="../../sigma.js/src/renderers/svg/sigma.svg.hovers.def.js"></script>
<script src="../../sigma.js/src/middlewares/sigma.middlewares.rescale.js"></script>
<script src="../../sigma.js/src/middlewares/sigma.middlewares.copy.js"></script>
<script src="../../sigma.js/src/misc/sigma.misc.animation.js"></script>
<script src="../../sigma.js/src/misc/sigma.misc.bindEvents.js"></script>
<script src="../../sigma.js/src/misc/sigma.misc.bindDOMEvents.js"></script>
<script src="../../sigma.js/src/misc/sigma.misc.drawHovers.js"></script>
<script src="../../sigma.js/src/middlewares/sigma.middlewares.rescale.js"></script>
<script src="../../sigma.js/src/middlewares/sigma.middlewares.copy.js"></script>
<script src="../../sigma.js/src/misc/sigma.misc.animation.js"></script>
<script src="../../sigma.js/src/misc/sigma.misc.bindEvents.js"></script>
<script src="../../sigma.js/src/misc/sigma.misc.bindDOMEvents.js"></script>
<script src="../../sigma.js/src/misc/sigma.misc.drawHovers.js"></script>
<script src="../../sigma.js/plugins/sigma.layout.forceAtlas2/worker.js"></script>
<script src="../../sigma.js/plugins/sigma.layout.forceAtlas2/supervisor.js"></script>
<script src="../../sigma.js/plugins/sigma.plugins.neighborhoods/sigma.plugins.neighborhoods.js"></script>
<script src="../../sigma.js/plugins/sigma.parsers.gexf/gexf-parser.js"></script>
<script src="../../sigma.js/plugins/sigma.parsers.gexf/sigma.parsers.gexf.js"></script>
''')

In [9]:
html_template = Template('''
<style>
  #graph-container {
      height: 800px;
  };
</style>
<div id="graph-container"></div>
<div id="message1"></div>
<script>$js_text</script>
''') 

In [13]:
js_text_tepmlate = Template('''

s = sigma.parsers.gexf(
    '$graph_data', 
    {
        container: 'graph-container', 
        settings:{
            drawLabels: false, 
        }
    }, 
    function(s){
        if (!(s.graph.neighbors)){
            sigma.classes.graph.addMethod('neighbors', function(nodeId) {
                var k,
                    neighbors = {},
                    index = this.allNeighborsIndex[nodeId] || {};

                for (k in index)
                  neighbors[k] = this.nodesIndex[k];

                return neighbors;
            })
        }
        else{
            document.getElementById('message1').innerHTML = "neighbors function already exists!";
        };
        
        s.graph.nodes().forEach(function(n){
            n.originalColor = n.color
        });

        s.graph.edges().forEach(function(e){
            e.originalColor = e.color
        });
        
        s.bind('clickStage', function(e){
            s.graph.nodes().forEach(function(n){
                n.color = n.originalColor;
            });
            s.graph.edges().forEach(function(n){
                n.color = n.originalColor;
            });
            s.refresh();
        });
        
        s.bind('clickNode', function(e){
            var nodeId = e.data.node.id, 
                toKeep = s.graph.neighbors(nodeId);
            document.getElementById('message1').innerHTML = nodeId;
            toKeep[nodeId] = e.data.node;
            s.graph.nodes().forEach(function(n){
                if(toKeep[n.id])
                    n.color = n.originalColor;
                else
                    n.color = '#eee';
            });
            s.graph.edges().forEach(function(n){
                if(toKeep[n.source] && toKeep[n.target])
                    n.color = n.originalColor;
                else
                    n.color = '#eee';
            });
            s.refresh();
        });
    }
);
''')

In [14]:
# G_3_2.gexf contains community of plaintiffs that cooperated with the same lawyer
js_text = js_text_tepmlate.substitute({
    "graph_data": 'G_3_2.gexf'
})
html_text = html_template.substitute({
    "js_text": js_text
})

In [15]:
HTML(html_text)